In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import joblib

import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [5]:
# 設定參數
LookBackNum = 12  # LSTM 往前看的筆數
ForecastNum = 48  # 預測筆數

# 載入訓練資料
DataName = os.getcwd() + '\ExampleTrainData(AVG)\AvgDATA_17.csv'
SourceData = pd.read_csv(DataName, encoding='utf-8')
SourceData.columns = ['Serial', 'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

# 特徵與目標
Regression_X_train = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values
Regression_y_train = SourceData[['Power(mW)']].values

# 正規化
LSTM_MinMaxModel = MinMaxScaler().fit(Regression_X_train)
Regression_X_train_norm = LSTM_MinMaxModel.transform(Regression_X_train)

# 構建 LSTM 訓練數據
X_train, y_train = [], []
for i in range(LookBackNum, len(Regression_X_train_norm)):
    X_train.append(Regression_X_train_norm[i - LookBackNum:i, :])
    y_train.append(Regression_X_train_norm[i, :])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))

# 建立 LSTM 模型
regressor = Sequential()
regressor.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=64, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=32))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=5, activation='relu'))  # 輸出為多個特徵

# 編譯模型
optimizer = Adam(learning_rate=0.001)
regressor.compile(optimizer=optimizer, loss='mean_squared_error')
regressor.fit(X_train, y_train, epochs=200, batch_size=64)

# 保存 LSTM 模型
regressor.save('three_LSTM_1.h5')

Epoch 1/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0581
Epoch 2/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0164
Epoch 3/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0134
Epoch 4/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0123
Epoch 5/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0101
Epoch 6/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0091
Epoch 7/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0084
Epoch 8/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0082
Epoch 9/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0080
Epoch 10/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0078
Epoch 11/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0070
Epoch 12/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0071
Epoch 13/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0068
Epoch 14/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0065
Epoch 15/200
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - lo

In [6]:
# 使用 LSTM 輸出作為基學習器特徵
lstm_features = regressor.predict(X_train)

# 確保 LSTM 特徵數量與回歸特徵對齊
lstm_features = lstm_features[:len(Regression_X_train_norm)]

# 堆疊模型
base_learners = [
    ('ridge', Ridge(alpha=1.0)),
    ('xgb', XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42))
]
stack_model = StackingRegressor(
    estimators=base_learners,
    final_estimator=LinearRegression()
)

# 融合特徵
stack_input = np.hstack([Regression_X_train_norm, lstm_features])

# 交叉驗證
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(stack_model, stack_input, Regression_y_train, cv=kf, scoring='r2')
print(f"交叉驗證 R² 平均分數: {np.mean(cv_scores):.4f}")
print(f"交叉驗證 R² 分數: {cv_scores}")

# 訓練堆疊模型
stack_model.fit(stack_input, Regression_y_train)

# 保存堆疊模型
joblib.dump(stack_model, 'stacking_model.h5')
print('Stacking Model Saved')

163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5202 and the array at index 1 has size 5190